## 为什么使用 Runnable

在使用大模型时，我们要考虑的事情很多，
基础的考虑包括流式输出、异步处理、Tokens统计，
更高级的部份还有记忆管理、工具回调等，
illufly 将这些封装为不同层次的类实现，而最基础的基类就是 Runnable。

### Runnable 基类实现了 `__call__` 方法

Runnable 实现了 `__call__` 方法，这个方法一般不需要重复实现。<br>
在 `__call__` 中自动选择调用子类必须实现的抽象函数 `call`。

在 python 语法中，一个类实现 `__call__` 方法的好处是，你就可以将类的实例当作方法一样使用。

In [1]:
from illufly.types import Runnable
Runnable.allowed_params()

{'name': 'Runnable 名称，默认为 {类名}.{id}',
 'handlers': 'EventBlock 迭代器处理函数列表，默认为 [log]，当调用 call 方法时，会使用该列表中的函数逐个处理 EventBlock',
 'threads_group': '如果由 illufly 管理线程池实现并发或异步，则可以指定线程组名称，默认为 DEFAULT',
 'providers': '实例的 consumer_dict 属性由 providers 列表中每个 Runnable 的 provider_dict 属性提供',
 'consumers': '实例的 provider_dict 属性将被 consumers 列表中每个 Runnable 引用',
 'dynamic_providers': '如果实例在不同周期中重复使用，可能会希望先在绑定前先清除旧的绑定，此时就应该使用动态绑定，即执行 bind_provider 时提供 dynamic=True 参数',
 'lazy_binding_map': '有时你无法确定被哪个对象绑定，但能确定绑定映射，此时就可以使用 lazy_binding_map 参数，在绑定时由对方根据该参数进行绑定'}

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        print("hi")

r = MyRun()
r()

hi


In [2]:
r

<MyRun.4415208560>

这样做有好处，也有限制。

**好处：**<br>
这很方便，也足够简单，因为你只需要记住 illufly 中的智能体对象只有一个方法，并且你不需要记住名字。

**限制：**<br>
似乎你的类只能有一个方法被调用。

illufly 框架一直在追求「保持简单」。因此，我们坚持使用这种机制，并提供其他解除限制的补救措施。

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [3]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


In [4]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

In [5]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")
        

r = MyRun()
r()

hi, illufly!

### 使用异步调用

In [8]:
from illufly.types import Runnable, EventBlock
from illufly.io import alog

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")
        

r = MyRun()
await r(handlers=[alog], verbose=True)

hi, illufly!

In [9]:
from illufly.types import Runnable, EventBlock
from illufly.io import alog

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

    async def async_call(*args, **kwargs):
        yield EventBlock("chunk", "I'm,")
        yield EventBlock("chunk", " a ")
        yield EventBlock("chunk", "async Runnable!")


r = MyRun()
await r(handlers=[alog], verbose=True)

I'm, a async Runnable!